In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [2]:
path = r"/Users/taanhtuan/Desktop/workproject/dot-detection/kddcup99_csv.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494015,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal
494016,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal
494017,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal
494018,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal


In [5]:
df.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'lnum_compromised', 'lroot_shell',
       'lsu_attempted', 'lnum_root', 'lnum_file_creations', 'lnum_shells',
       'lnum_access_files', 'lnum_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'label'],
      dtype='object')

In [6]:
len(df.columns)

42

In [7]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [8]:
num_of_samples = len(df)
print(num_of_samples)
for i, col in enumerate(df.columns):
    print(i, ": ", col)
    unique_values = df[col].value_counts()
    unique_values = unique_values.reset_index()
    for value in unique_values.iterrows():
        percentage = float((value[1][col]*100/num_of_samples))
        if percentage > 0.01:
            print("the percentage of ", value[1]['index'], " occupied", percentage)
    print("and some values with trivial percentage")

494020
0 :  duration
the percentage of  0  occupied 97.50010121047731
the percentage of  1  occupied 0.5011942836322416
the percentage of  2  occupied 0.17610623051698313
the percentage of  3  occupied 0.12651309663576374
the percentage of  5  occupied 0.11214120885794097
the percentage of  2630  occupied 0.1004007934901421
the percentage of  4  occupied 0.08359985425691267
the percentage of  14  occupied 0.06517954738674547
the percentage of  10  occupied 0.039269665195741064
the percentage of  7  occupied 0.034209141330310514
the percentage of  6  occupied 0.03178008987490385
the percentage of  9  occupied 0.02449293550868386
the percentage of  8  occupied 0.020849358325573865
the percentage of  15  occupied 0.02064693737095664
the percentage of  12  occupied 0.020242095461722197
the percentage of  2715  occupied 0.020039674507104976
the percentage of  11  occupied 0.017813044006315535
the percentage of  2620  occupied 0.017003360187846647
the percentage of  2625  occupied 0.01659851

the percentage of  241  occupied 0.07489575320837213
the percentage of  287  occupied 0.07347880652605157
the percentage of  294  occupied 0.07327638557143436
the percentage of  300  occupied 0.07307396461681713
the percentage of  288  occupied 0.07226428079834825
the percentage of  319  occupied 0.07206185984373102
the percentage of  292  occupied 0.07145459697987935
the percentage of  314  occupied 0.07125217602526214
the percentage of  316  occupied 0.07125217602526214
the percentage of  322  occupied 0.06963280838832436
the percentage of  242  occupied 0.06922796647908991
the percentage of  201  occupied 0.06922796647908991
the percentage of  324  occupied 0.0690255455244727
the percentage of  289  occupied 0.06882312456985547
the percentage of  240  occupied 0.0682158617060038
the percentage of  18  occupied 0.0682158617060038
the percentage of  321  occupied 0.06740617788753492
the percentage of  313  occupied 0.06740617788753492
the percentage of  199  occupied 0.067203756932917

and some values with trivial percentage
5 :  dst_bytes
the percentage of  0  occupied 82.6399740901178
the percentage of  105  occupied 0.900975669001255
the percentage of  147  occupied 0.5062548074976722
the percentage of  146  occupied 0.4633415651188211
the percentage of  8314  occupied 0.43176389619853445
the percentage of  145  occupied 0.19938464029796366
the percentage of  42  occupied 0.18642969920246144
the percentage of  330  occupied 0.17286749524310757
the percentage of  329  occupied 0.16274644751224646
the percentage of  331  occupied 0.16051981701145704
the percentage of  332  occupied 0.14493340350593092
the percentage of  328  occupied 0.13582446054815595
the percentage of  333  occupied 0.12286951945265374
the percentage of  327  occupied 0.11902352131492652
the percentage of  334  occupied 0.11376057649487875
the percentage of  46  occupied 0.10546131735557265
the percentage of  48  occupied 0.07975385611918545
the percentage of  335  occupied 0.07631269989069268
th

the percentage of  314  occupied 0.012145257277033318
the percentage of  1875  occupied 0.012145257277033318
the percentage of  388  occupied 0.012145257277033318
the percentage of  4841  occupied 0.012145257277033318
the percentage of  810  occupied 0.011942836322416097
the percentage of  277  occupied 0.011942836322416097
the percentage of  275  occupied 0.011942836322416097
the percentage of  757  occupied 0.011942836322416097
the percentage of  765  occupied 0.011942836322416097
the percentage of  39  occupied 0.011740415367798875
the percentage of  303  occupied 0.011740415367798875
the percentage of  412  occupied 0.011740415367798875
the percentage of  288  occupied 0.011740415367798875
the percentage of  482  occupied 0.011740415367798875
the percentage of  290  occupied 0.011740415367798875
the percentage of  807  occupied 0.011537994413181653
the percentage of  341  occupied 0.011537994413181653
the percentage of  1481  occupied 0.011537994413181653
the percentage of  2543  o

the percentage of  250  occupied 0.1408849844135865
the percentage of  282  occupied 0.14048014250435206
the percentage of  236  occupied 0.14048014250435206
the percentage of  232  occupied 0.1400753005951176
the percentage of  453  occupied 0.13926561677664873
the percentage of  22  occupied 0.1390631958220315
the percentage of  254  occupied 0.13825351200356262
the percentage of  230  occupied 0.1380510910489454
the percentage of  274  occupied 0.13764624913971094
the percentage of  226  occupied 0.13764624913971094
the percentage of  260  occupied 0.13683656532124205
the percentage of  149  occupied 0.1362293024573904
the percentage of  481  occupied 0.13440751386583538
the percentage of  256  occupied 0.13400267195660095
the percentage of  280  occupied 0.1335978300473665
the percentage of  228  occupied 0.13278814622889762
the percentage of  276  occupied 0.13238330431966316
the percentage of  266  occupied 0.13238330431966316
the percentage of  284  occupied 0.13197846241042874


...

the percentage of  0.2  occupied 0.013967045868588317
the percentage of  0.18  occupied 0.013562203959353872
the percentage of  0.16  occupied 0.012752520140884984
the percentage of  0.19  occupied 0.01234767823165054
the percentage of  0.02  occupied 0.012145257277033318
the percentage of  0.22  occupied 0.010930731549329987
the percentage of  0.21  occupied 0.010121047730861098
and some values with trivial percentage
30 :  srv_diff_host_rate
the percentage of  0.0  occupied 92.98753086919558
the percentage of  1.0  occupied 1.6394073114448808
the percentage of  0.12  occupied 0.30525079956277074
the percentage of  0.5  occupied 0.2870329136472208
the percentage of  0.67  occupied 0.28541354601028296
the percentage of  0.33  occupied 0.2514068256345897
the percentage of  0.11  occupied 0.24432209222298693
the percentage of  0.25  occupied 0.22671146917128862
the percentage of  0.1  occupied 0.21578073762195862
the percentage of  0.14  occupied 0.21152989757499696
the percentage of  0.

the percentage of  208  occupied 0.022063884053277194
the percentage of  183  occupied 0.021861463098659974
the percentage of  179  occupied 0.021861463098659974
the percentage of  197  occupied 0.021861463098659974
the percentage of  171  occupied 0.021861463098659974
the percentage of  210  occupied 0.02165904214404275
the percentage of  204  occupied 0.02165904214404275
the percentage of  192  occupied 0.02165904214404275
the percentage of  166  occupied 0.02145662118942553
the percentage of  195  occupied 0.02125420023480831
the percentage of  207  occupied 0.021051779280191085
the percentage of  193  occupied 0.021051779280191085
the percentage of  202  occupied 0.020849358325573865
the percentage of  181  occupied 0.020849358325573865
the percentage of  189  occupied 0.02064693737095664
the percentage of  205  occupied 0.02064693737095664
the percentage of  218  occupied 0.02044451641633942
the percentage of  212  occupied 0.020242095461722197
the percentage of  176  occupied 0.0

the percentage of  187  occupied 0.022266305007894418
the percentage of  112  occupied 0.022266305007894418
the percentage of  97  occupied 0.022266305007894418
the percentage of  220  occupied 0.022063884053277194
the percentage of  113  occupied 0.022063884053277194
the percentage of  141  occupied 0.022063884053277194
the percentage of  155  occupied 0.022063884053277194
the percentage of  71  occupied 0.022063884053277194
the percentage of  144  occupied 0.021861463098659974
the percentage of  142  occupied 0.021861463098659974
the percentage of  103  occupied 0.021861463098659974
the percentage of  154  occupied 0.021861463098659974
the percentage of  84  occupied 0.021861463098659974
the percentage of  153  occupied 0.02145662118942553
the percentage of  94  occupied 0.02145662118942553
the percentage of  145  occupied 0.02145662118942553
the percentage of  106  occupied 0.02145662118942553
the percentage of  119  occupied 0.02125420023480831
the percentage of  51  occupied 0.021

the percentage of  0.79  occupied 0.010121047730861098
and some values with trivial percentage
35 :  dst_host_same_src_port_rate
the percentage of  1.0  occupied 58.47597263268693
the percentage of  0.0  occupied 28.917655155661713
the percentage of  0.01  occupied 4.435447957572568
the percentage of  0.02  occupied 1.4630986599732805
the percentage of  0.03  occupied 0.8944981984535039
the percentage of  0.04  occupied 0.5564552042427432
the percentage of  0.05  occupied 0.440467997247075
the percentage of  0.06  occupied 0.36111898303712403
the percentage of  0.5  occupied 0.3323752074814785
the percentage of  0.08  occupied 0.289057123193393
the percentage of  0.33  occupied 0.28602080887413467
the percentage of  0.07  occupied 0.2858183879195174
the percentage of  0.25  occupied 0.2578842961823408
the percentage of  0.2  occupied 0.2329865187644225
the percentage of  0.17  occupied 0.20930326707420752
the percentage of  0.11  occupied 0.19331201165944698
the percentage of  0.12  oc

the percentage of  0.93  occupied 0.020039674507104976
the percentage of  0.07  occupied 0.019027569734018864
the percentage of  0.89  occupied 0.01740820209708109
the percentage of  0.8  occupied 0.01538399255090887
the percentage of  0.78  occupied 0.014979150641674426
the percentage of  0.91  occupied 0.013764624913971095
the percentage of  0.74  occupied 0.012752520140884984
the percentage of  0.92  occupied 0.01234767823165054
the percentage of  0.85  occupied 0.012145257277033318
the percentage of  0.87  occupied 0.010525889640095543
the percentage of  0.82  occupied 0.010525889640095543
the percentage of  0.75  occupied 0.01032346868547832
and some values with trivial percentage
41 :  label
the percentage of  smurf  occupied 56.83777984696976
the percentage of  neptune  occupied 21.699728755920813
the percentage of  normal  occupied 19.690903202299502
the percentage of  back  occupied 0.44593336302174
the percentage of  satan  occupied 0.32164689688676573
the percentage of  ipsw

In [9]:
onehot_columns = ['protocol_type', 'service', 'flag']
dropped_columns = ['land', 'wrong_fragment', 'lsu_attempted']

In [10]:
def onehot_encoding(df, colname):
    print("Encode", colname)
    dummies = pd.get_dummies(df[colname])
    print("len: ", len(dummies.columns))
    for x in dummies.columns:
        dummy_name = f"{colname}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(colname, axis=1, inplace=True)

In [11]:
df = pd.read_csv(path)
for colname in onehot_columns:
    onehot_encoding(df,colname)

Encode protocol_type
len:  3
Encode service
len:  66
Encode flag
len:  11


In [12]:
df.fillna(-999, inplace=True)

In [13]:
for col in df.columns:
    print(col, " : ", df[col][1])

duration  :  0
src_bytes  :  239
dst_bytes  :  486
land  :  0
wrong_fragment  :  0
urgent  :  0
hot  :  0
num_failed_logins  :  0
logged_in  :  1
lnum_compromised  :  0
lroot_shell  :  0
lsu_attempted  :  0
lnum_root  :  0
lnum_file_creations  :  0
lnum_shells  :  0
lnum_access_files  :  0
lnum_outbound_cmds  :  0
is_host_login  :  0
is_guest_login  :  0
count  :  8
srv_count  :  8
serror_rate  :  0.0
srv_serror_rate  :  0.0
rerror_rate  :  0.0
srv_rerror_rate  :  0.0
same_srv_rate  :  1.0
diff_srv_rate  :  0.0
srv_diff_host_rate  :  0.0
dst_host_count  :  19
dst_host_srv_count  :  19
dst_host_same_srv_rate  :  1.0
dst_host_diff_srv_rate  :  0.0
dst_host_same_src_port_rate  :  0.05
dst_host_srv_diff_host_rate  :  0.0
dst_host_serror_rate  :  0.0
dst_host_srv_serror_rate  :  0.0
dst_host_rerror_rate  :  0.0
dst_host_srv_rerror_rate  :  0.0
label  :  normal
protocol_type-icmp  :  0
protocol_type-tcp  :  1
protocol_type-udp  :  0
service-IRC  :  0
service-X11  :  0
service-Z39_50  :  0
se

In [14]:
label_kinds = df["label"].value_counts()
print(len(label_kinds))
for label in label_kinds.iteritems():
    print(label)

23
('smurf', 280790)
('neptune', 107201)
('normal', 97277)
('back', 2203)
('satan', 1589)
('ipsweep', 1247)
('portsweep', 1040)
('warezclient', 1020)
('teardrop', 979)
('pod', 264)
('nmap', 231)
('guess_passwd', 53)
('buffer_overflow', 30)
('land', 21)
('warezmaster', 20)
('imap', 12)
('rootkit', 10)
('loadmodule', 9)
('ftp_write', 8)
('multihop', 7)
('phf', 4)
('perl', 3)
('spy', 2)


In [15]:
df['label'] = df['label'].replace(['spy','perl','phf','multihop','ftp_write','loadmodule','rootkit'],'other')

In [16]:
label_kinds = df["label"].value_counts()
print(len(label_kinds))
for label in label_kinds.iteritems():
    print(label)

17
('smurf', 280790)
('neptune', 107201)
('normal', 97277)
('back', 2203)
('satan', 1589)
('ipsweep', 1247)
('portsweep', 1040)
('warezclient', 1020)
('teardrop', 979)
('pod', 264)
('nmap', 231)
('guess_passwd', 53)
('other', 43)
('buffer_overflow', 30)
('land', 21)
('warezmaster', 20)
('imap', 12)


In [17]:
ord_enc = OrdinalEncoder()
df["label"] = ord_enc.fit_transform(df[["label"]])

In [18]:
X = df.drop(["label"], axis=1)
Y = df["label"]

In [19]:
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,...,flag-REJ,flag-RSTO,flag-RSTOS0,flag-RSTR,flag-S0,flag-S1,flag-S2,flag-S3,flag-SF,flag-SH
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
print(type(X))
print(type(Y))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [21]:
x_train, x_val, y_train, y_val = train_test_split(X, Y,test_size=0.33, random_state=42)

In [22]:
print("X_train.shape", x_train.shape)
print("X_validation.shape", x_val.shape)
print("y_train.shape", y_train.shape)
print("y_validation.shape", y_val.shape)

X_train.shape (330993, 118)
X_validation.shape (163027, 118)
y_train.shape (330993,)
y_validation.shape (163027,)


In [23]:
estimator = lgb.LGBMClassifier(
                        learning_rate=0.125,
                        metric='multi_logloss',
                        n_estimators=20,
                        num_leaves=38
                        )
param_grid = {
            'n_estimators': [x for x in range(24, 40, 2)],
            'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}

gridsearch = GridSearchCV(estimator, param_grid)

In [ ]:
gridsearch.fit(x_train, y_train,
                       eval_set=[(x_val, y_val)],
                       eval_metric=['multi_logloss', 'auc_mu'],
                       early_stopping_rounds=10)

[1]	valid_0's auc_mu: 0.994915	valid_0's multi_logloss: 0.759564
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc_mu: 0.697696	valid_0's multi_logloss: 0.597568
[3]	valid_0's auc_mu: 0.849495	valid_0's multi_logloss: 0.550886
[4]	valid_0's auc_mu: 0.66466	valid_0's multi_logloss: 0.601396
[5]	valid_0's auc_mu: 0.586688	valid_0's multi_logloss: 0.661141
[6]	valid_0's auc_mu: 0.646634	valid_0's multi_logloss: 0.418583
[7]	valid_0's auc_mu: 0.612887	valid_0's multi_logloss: 0.429971
[8]	valid_0's auc_mu: 0.580873	valid_0's multi_logloss: 0.350957
[9]	valid_0's auc_mu: 0.5683	valid_0's multi_logloss: 0.478961
[10]	valid_0's auc_mu: 0.576672	valid_0's multi_logloss: 0.370382
[11]	valid_0's auc_mu: 0.558688	valid_0's multi_logloss: 0.682186
Early stopping, best iteration is:
[1]	valid_0's auc_mu: 0.994915	valid_0's multi_logloss: 0.759564
[1]	valid_0's auc_mu: 0.994195	valid_0's multi_logloss: 0.752779
Training until validation scores don't improve for 10 rounds

[5]	valid_0's auc_mu: 0.563643	valid_0's multi_logloss: 0.410342
[6]	valid_0's auc_mu: 0.635539	valid_0's multi_logloss: 0.417097
[7]	valid_0's auc_mu: 0.600732	valid_0's multi_logloss: 0.450062
[8]	valid_0's auc_mu: 0.640502	valid_0's multi_logloss: 0.464309
[9]	valid_0's auc_mu: 0.574833	valid_0's multi_logloss: 0.515222
[10]	valid_0's auc_mu: 0.61635	valid_0's multi_logloss: 0.396263
[11]	valid_0's auc_mu: 0.578431	valid_0's multi_logloss: 0.505684
Early stopping, best iteration is:
[1]	valid_0's auc_mu: 0.993947	valid_0's multi_logloss: 0.758408
[1]	valid_0's auc_mu: 0.994915	valid_0's multi_logloss: 0.759564
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc_mu: 0.697698	valid_0's multi_logloss: 0.597568
[3]	valid_0's auc_mu: 0.849496	valid_0's multi_logloss: 0.550886
[4]	valid_0's auc_mu: 0.664661	valid_0's multi_logloss: 0.601396
[5]	valid_0's auc_mu: 0.58599	valid_0's multi_logloss: 0.659233
[6]	valid_0's auc_mu: 0.643896	valid_0's multi_logloss: 0.4

In [ ]:
Y_pred = gridsearch.predict_proba(X_test)[:, 1]